<a href="https://colab.research.google.com/github/Coopyrightdmin/Text-Summarization/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Summarization

based on the following article: https://stackabuse.com/text-summarization-with-nltk-in-python/ with many bugs and not very performant.

In [47]:
import bs4 as bs  
import urllib.request  
import re
import heapq  

import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

# Dataset
uri='https://fr.wikipedia.org/wiki/Napol%C3%A9on_Ier'

# Retrieving text from Html paragraphs
scraped_data = urllib.request.urlopen(uri)  
article = scraped_data.read()
parsed_article = bs.BeautifulSoup(article,'lxml')
paragraphs = parsed_article.find_all('p')

def summarize(article_text, val):
  summary_sentences = []
    
  # Removing Square Brackets and Extra Spaces
  article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)  
  article_text = re.sub(r'\s+', ' ', article_text)  

  # Removing special characters and digits
  formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )  
  formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)  

  if not len(formatted_article_text) <= 1: 

    # Tokenizing content
    sentence_list = nltk.sent_tokenize(article_text)  
    stopwords = nltk.corpus.stopwords.words('french')

    word_frequencies = {}  
    for word in nltk.word_tokenize(formatted_article_text):  
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1

    maximum_frequncy = max(word_frequencies.values())

    for word in word_frequencies.keys():  
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

    sentence_scores = {}  
    for sent in sentence_list:  
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]

    summary_sentences = heapq.nlargest(val, sentence_scores, key=sentence_scores.get)
  return summary_sentences

# 1st Loop to summarize paragraphs
tmp = []
for p in paragraphs:  
  s = summarize(p.text, 1)
  tmp = tmp + s

# 2nd Loop to select sentences
summary = ""
print('\n'.join(summarize('\n'.join(tmp), 15)))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Poussés par les Britanniques et les Turcs, les mamelouks survivants influencent la population du Caire qui se révolte le 21 octobre contre les Français.
Il pousse aussi au développement des usines de coton, installées dans les biens nationaux, alors que les guerres ont suscité un besoin de textiles pour habiller les armées.
Bien que les Russes abandonnent le terrain, les pertes sont presque équivalentes dans les deux camps.
Napoléon retourne à plusieurs reprises en Corse, où les luttes de clans avaient repris, les paolistes soutenant la monarchie à l’anglaise, et les Bonaparte la Révolution.
Napoléon veut montrer qu’il est le fondateur de la « quatrième dynastie », celle des Bonaparte, après les Mérovingiens, les Carolingiens, et les Capétiens.
Pourtant formé 

# TextRank Algorithm
refering to the following article: https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/
You can download the dataset we’ll be using from [link text](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/10/tennis_articles_v4.csv)



In [0]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import bs4 as bs  
import urllib.request  

df = ['https://fr.wikipedia.org/wiki/Napol%C3%A9on_Ier']

# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
  
sentences = []
for s in df:
  print(s)
  scraped_data = urllib.request.urlopen(s)  
  article = scraped_data.read()
  parsed_article = bs.BeautifulSoup(article,'lxml')
  paragraphs = parsed_article.find_all('p')
  article_text = ""
  for p in paragraphs:  
    article_text += p.text
    
  # Removing Square Brackets and Extra Spaces
  article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)  
  article_text = re.sub(r'\s+', ' ', article_text)  
  sentences.append(sent_tokenize(article_text))

sentences = [y for x in sentences for y in x] # flatten list

# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)
  
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
      
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

# Extract top 10 sentences as the summary
for i in range(10):
  print(ranked_sentences[i][1])

https://fr.wikipedia.org/wiki/Napol%C3%A9on_Ier
Lacrosse et son successeur Ernouf réintroduisent progressivement l'esclavage sous la forme d'un travail forcé, puis de l'esclavage lui-même par le biais d'un arrêté de police rural du 22 avril 1803 qui fait référence à certains articles du Code noir, et enfin avec l'adoption du décret du Premier consul, daté du 16 juillet 1802, qui énonce que « La Colonie de la Guadeloupe et Dépendances sera régie, à l’instar de La Martinique, de Ste-Lucie, de Tabago, et des Colonies orientales, par les mêmes loix qui y étoient en vigueur en 1789 ».
Le général noir est gouverneur général de la colonie au nom de la France depuis 1797 et il est suspecté de connivences avec les États-Unis avec lesquels, au mépris du principe de l'Exclusif, il commerce ouvertement depuis que la prospérité est revenue.
La mort de Napoléon résulterait donc de la combinaison d'une hépatite chronique ancienne, d'un ulcère de l'estomac lié au stress évoluant jusqu'à la perforation

# Lsa Summarizer

In [45]:
# -*- coding: utf-8 -*-

from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
import re

LANGUAGE = "french"
SENTENCES_COUNT = 15


if __name__ == "__main__":
    url = "https://fr.wikipedia.org/wiki/Napol%C3%A9on_Ier"
    parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)

    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)

    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        print(sentence)

Le 28 août 1793, alors que Marseille vient d'être reprise par les Républicains jacobins et que la famille Bonaparte s'y installe, Toulon , tenue par les fédéralistes et les royalistes, se livre aux troupes britanniques et espagnoles.
Le calme revient et Bonaparte rétablit la situation en décrétant finalement une amnistie générale, non sans avoir fait couper bon nombre de têtes, exhibées à la foule terrorisée, et canonner la grande mosquée Al-Azhar .
Les révisionnistes avaient envisagé une démission collective des cinq directeurs , mais les assemblées ont du retard car cette idée ne fait pas l’unanimité ; Bonaparte s’impatiente et décide d’intervenir.
Mais Bonaparte reste attaché aux formes juridiques et, dans la soirée du 19 brumaire, les députés restent à Saint-Cloud pour voter la décision de nommer deux commissions pour préparer une nouvelle constitution.
Napoléon, qui se veut rassembleur, décide d’associer aux symboles de son règne les images qui ont pu représenter auparavant la Fra

# Lex Rank

In [58]:
# -*- coding: utf-8 -*-

from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words


LANGUAGE = "french"
SENTENCES_COUNT = 15


if __name__ == "__main__":
    url = "https://fr.wikipedia.org/wiki/Napol%C3%A9on_Ier"
    parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
  
    stemmer = Stemmer(LANGUAGE)

    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)

    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        print(sentence)

Second enfant de Charles Bonaparte et Letizia Ramolino , Napoléon Bonaparte est un militaire, général dans les armées de la Première République française, née de la Révolution , commandant en chef de l'armée d'Italie puis de l'armée d'Orient.
De plus, il ne reste que 14 officiers sur 80 dans son régiment, le 4 e d'artillerie.
L'arsenal, la flotte française est livrée aux Anglais.
Celui-ci ne l'écoute pas et ne suit pas ses conseils [23] .
Il est à l'origine de la .
Il ne s’agit plus pour lui de livrer une guerre de siège à l’aide de 30 000 à 50 000 hommes, mais de rechercher la bataille décisive, engageant plus de 100 000 hommes s’il le faut.
Il déclare à Caulaincourt : « Qu’on a de peine à mourir, qu’on est malheureux d’avoir une constitution qui repousse la fin d’une vie qu’il me tarde tant de voir finir !
une première fois le 9 mars 1796 avec Joséphine de Beauharnais , qui est ensuite couronnée impératrice ; ce mariage restant sans enfants, il se conclut par un divorce, prononcé par

# Reduction

In [61]:
# -*- coding: utf-8 -*-

from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words


LANGUAGE = "french"
SENTENCES_COUNT = 15


if __name__ == "__main__":
    url = "https://fr.wikipedia.org/wiki/Napol%C3%A9on_Ier"
    parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
  
    stemmer = Stemmer(LANGUAGE)

    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)

    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        print(sentence)

Titre Empereur des Français 20mars – 22juin1815 (3 mois et 2 jours) Prédécesseur Louis XVIII (roi de France)Successeur Napoléon II (prétendant) Louis XVIII (roi de France) 18mai1804 – 6avril1814 (9 ans, 10 mois et 19 jours) Couronnement 2décembre1804 , en la cathédrale Notre-Dame de Paris Prédécesseur Lui-même ( Premier consul de la République )Successeur Louis XVIII (roi de France) Roi d'Italie 17mars1805 – 11avril1814 (9 ans et 25 jours) Prédécesseur Lui-même (président de la République italienne)Successeur Victor-Emmanuel II (roi d'Italie en 1861) Protecteur de la Confédération du Rhin 12juillet1806 – 19octobre1813 (7 ans, 3 mois et 7 jours) Prédécesseur François II ( Empereur des Romains ) Successeur Confédération germaniqueMédiateur de la Confédération suisse 19février1803 – 19octobre1813 (10 ans et 8 mois) Prédécesseur République helvétique Successeur Confédération des XXII cantonsPrésident de la République italienne Napoléon Bonaparte 26janvier1802 – 17mars1805 (3 ans, 1 mois et

# pyTeaser

In [67]:
!pip install pyteaser 
from pyteaser import SummarizeUrl

url = 'https://fr.wikipedia.org/wiki/Napol%C3%A9on_Ier'
summaries = SummarizeUrl(url)
print (summaries)

  Using cached https://files.pythonhosted.org/packages/d4/7a/310592c6e7998440e56a8650446ecf3ded076431415c60f0f3b946b54462/pyteaser-2.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/7b/44/25b7283e50585f0b4156960691d951b05d061abf4a714078393e51929b30/cssselect-1.0.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/1e/ee/295988deca1a5a7accd783d0dfe14524867e31abb05b6c0eeceee49c759d/BeautifulSoup-3.2.1.tar.gz
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/tmp/pip-install-5df9hvdj/beautifulsoup/setup.py", line 22
        print "Unit tests have failed!"
                                      ^
    SyntaxError: Missing parentheses in call to 'print'. Did you mean print("Unit tests have failed!")?
    
    ----------------------------------------
Command "python setup.py egg_info" failed with error code 1 in /tmp/pip-install-5df9hvdj/beautifulsoup/


ModuleNotFoundError: ignored